# Assignment 09: Final Project
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023

# Objectives

Upon completion of this assignment, students will have:

1. Described a list generation process in detail
2. Described the experiment details
3. Stated two new new hypotheses/questions to test
3. Visualized processed data testing these hypotheses
4. Performed a statistical analyses to test the hypotheses
5. Summarized the results

# Assignment

Write text (in MarkDown cells) and code (in Code cells) in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A10_Final_Project_mst3k).


## Details

The goal of the final project is to synthesize material covered in the class and produce part of what would go into an actual scientific publication based on *one or more* of the experiments we ran in the class. Specifically, you will be writing part of the Methods and Results sections.

You can copy the basic template code for loading and processing the data from the class lessons. We also outline what each section of your assignment should include. In addition to describing the experiment in detail, you will be developing novel hypotheses we did not test in class and then performing the associated analyses and visualizing the results. As always, make sure to label all figures and be sure to refer to the code in the lesson notebooks as a guide for your analyses.

Please feel free to reach out to us on Discord if you have any questions along the way. For example, we're happy to weigh in on whether or not your hypotheses are novel.


* ***If you select the project in this notebook, when you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVA Canvas.***  

## General Imports

In [3]:
# import some useful libraries
import random
import csv
from copy import deepcopy
import pickle
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as smc
from statsmodels.stats.api import anova_lm
import matplotlib.pyplot as plt
import seaborn as sns


from smile.common import *
from smile.scale import scale as s

from smile.common import *
from smile.scale import scale as s
from smile.math_distract import MathDistract
from smile.moving_dots import MovingDots
import plotnine as pn
import scipy.stats.distributions as dists     # probability distributions
from scipy import stats
from glob import glob
import os
import arviz as az
import bambi as bmb

from smile.log import log2dl

from ci_within import ci_within

ModuleNotFoundError: No module named 'ci_within'

# Hypotheses

1. I hypothesize that words that are presented in the middle of the test are more likely to be forgotten than the first three words presented and the last three words presented. 
> Serial position effect: Assess whether participants show a tendency to remember items at the beginning (primacy effect) or end (recency effect) of the list more accurately. This can provide insights into how the position of an item in a sequence affects memory recall.
2. I hypothesize that if the time difference between the study word presented from the test word is shorter, the participant will have a greater recall and accuracy than if the time series between the study list and test list is greater. 
> Retention over time: Test participants' ability to retain information over different time intervals. This can include short-term memory (immediate recall) and long-term memory (recall after a delay).

# Methods

## List generation

*Provide enough detail (in words) about the list generation so that someone could recreate the list generation code themselves. Be sure to state all the specific parameters used, including number of stimuli per block and number of blocks. Refer to the list generation code we provided to find all the information you need.*

## Refreshing Valence Study

The main question of this study is whether recognition memory for
words depends on the emotional or affective valence of those words and whether there is an interaction between attention refreshing and valence.

Participants will study lists of positive (+), negative (-), and
neutral (~) words and then, after a short delay, they will be given a
recognition test over all the studied target words plus a matched set
of non-studied lures.  The stimuli are contained in three separate CSV
files:

- [Positive Pool](./pos_pool.csv)
- [Negative Pool](./neg_pool.csv)
- [Neutral Pool](./neu_pool.csv)

You will need to read these files in as lists of dictionaries (hint,
use the ``DictReader`` from the ``csv`` module that was covered in
class.)  

Use these pools to create lists with trials of valence crossed with three experimental conditions:

1. *Repeated*: Where a word will be immediately repeated as the next word.
2. *Refreshed*: Where you will indicate the participant should "refresh" the previous word by presenting a "+".
3. *Once-presented*: Where a word is only presented once and is *not* repeated or refreshed.

We suggest that you generate the study items for a list in two stages. In the first stage you shuffle all combinations of the trial types (Valence crossed with Condition). In the second stage you loop over those conditions and append trials to a block depending on the information in each trial type. For the Repeated and Refreshed you would append two items, for the Once-presented you would only append one.

You will need to generate a matching test list for each study list
that includes all the studied items, plus a set of lures that match
the valence of the studied words.

Be sure to add in information to each trial dictionary that identifies
the word, its valence, the condition of that trial, and whether it is a
target or a lure.  Feel free to add in more information if you would
like.

In [3]:
# Code to read in the pools
def read_and_shuffle(pool_file):
    """Read in and shuffle a pool."""
    # create a dictionary reader
    dr = csv.DictReader(open(pool_file, 'r'))

    # read in all the lines into a list of dicts
    pool = [l for l in dr]

    # shuffle it so that the we get new items each time
    random.shuffle(pool)
    
    # report out some pool info
    print(pool_file, len(pool))

    # return the shuffled pool
    return pool

In creating the list generation, the researcher will first have to code a function. This is done by creating a definition or def name_of_function(x): create a dictionary reader, read in all the lines into a list of dictionaries, shuffle it so that we get new items each time, report out some pool information (by taking the length or len of the pool file, and finally return the shuffled pool. 

In [4]:
# Configuration Section
pool_files = {'pos': 'pos_pool.csv',
              'neg': 'neg_pool.csv',
              'neu': 'neu_pool.csv'}

rep_conds = ['once', 'repeat', 'refresh']
val_conds = ['pos', 'neg', 'neu']

# what to show for refreshed items
ref_text = '+'

num_reps = 1
num_blocks = 1

In the next section of code, the researcher will define all of the terms for the rest of the experiment. You will first create a dictionary that allows for pos to represent the csv file that contains the positive word pool. Next, the researcher will define the valence conditions. For example, val_cond is given the attributed pos, neg, or neu. They also define what the repition conditions are and what should appear when a word is refreshed (+). The differing repition condtions should be once, repeated, or refreshed. It also is defining the number of reps as one and the number of blocks as one. These numbers for the number of reps and the number of blocks can easily be changed for anyone wanting to reproduce the study.

In [5]:
# read all the pools into a dictionary
pools = {val: read_and_shuffle(pool_files[val])
         for val in val_conds}

# show the first 5 items of the pos pool
pools['pos'][:5]

pos_pool.csv 301
neg_pool.csv 292
neu_pool.csv 208


[{'description': 'leader',
  'word_no': '844',
  'valence_mean': '7.6299999999999999',
  'valence_sd': '1.5900000000000001',
  'arousal_mean': '6.2699999999999996',
  'arousal_sd': '2.1800000000000002',
  'dominance_mean': '7.8799999999999999',
  'dominance_sd': '1.6000000000000001',
  'word_frequency': '74'},
 {'description': 'restaurant',
  'word_no': '960',
  'valence_mean': '6.7599999999999998',
  'valence_sd': '1.8500000000000001',
  'arousal_mean': '5.4100000000000001',
  'arousal_sd': '2.5499999999999998',
  'dominance_mean': '5.7300000000000004',
  'dominance_sd': '1.4099999999999999',
  'word_frequency': '41'},
 {'description': 'vision',
  'word_no': '480',
  'valence_mean': '6.6200000000000001',
  'valence_sd': '1.8400000000000001',
  'arousal_mean': '4.6600000000000001',
  'arousal_sd': '2.4300000000000002',
  'dominance_mean': '6.0199999999999996',
  'dominance_sd': '1.96',
  'word_frequency': '56'},
 {'description': 'silly',
  'word_no': '981',
  'valence_mean': '7.4100000

In [6]:
# create the conds
# fully crossed with all combos of val and rep
conds = []
for val in val_conds:
    for rep in rep_conds:
        # I decided to call the repetition condition cond
        conds.append({'valence': val, 'cond': rep})
conds

[{'valence': 'pos', 'cond': 'once'},
 {'valence': 'pos', 'cond': 'repeat'},
 {'valence': 'pos', 'cond': 'refresh'},
 {'valence': 'neg', 'cond': 'once'},
 {'valence': 'neg', 'cond': 'repeat'},
 {'valence': 'neg', 'cond': 'refresh'},
 {'valence': 'neu', 'cond': 'once'},
 {'valence': 'neu', 'cond': 'repeat'},
 {'valence': 'neu', 'cond': 'refresh'}]

In this section of code, the researcher is producing all the viable configurations of a word, it's repitition, and it's valence. For example, it could be a positive word, repeated.  

In [7]:
# make a function for generating a block
# with a study and test list
def make_block():
    """Generate a block, uses global variables"""
    # loop and create the repeated conditions
    block_conds = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        block_conds.extend(deepcopy(conds))

    # shuffle the conds for that block
    random.shuffle(block_conds)

    # loop over block conds and add items to study/test lists
    study_list = []
    test_list = []
    for cond in block_conds:
        # use the valence to grab study and test items
        study_item = pools[cond['valence']].pop()
        test_item = pools[cond['valence']].pop()

        # update with the cond info
        study_item.update(cond)
        test_item.update(cond)

        # add in relevant info for study and test
        study_item['pres_num'] = 1
        study_item['type'] = 'target'
        test_item['type'] = 'lure'
        test_item['pres_num'] = 1   # just so the keys match

        # append them to the respective lists
        # study item is added to both study and test
        study_list.append(study_item)
        test_list.append(study_item)
        test_list.append(test_item)

        # process the repetition and refresh conditions
        if cond['cond'] in ['refresh', 'repeat']:
            # copy the study item
            rep_item = deepcopy(study_item)

            # modify required values
            rep_item['pres_num'] = 2

            # change the description if refreshing
            if cond['cond'] == 'refresh':
                rep_item['description'] = ref_text

            # append it to the study list
            study_list.append(rep_item)
            
    # must shuffle the test list
    random.shuffle(test_list)
    
    # make a dictionary to return
    block = {'study': study_list, 'test': test_list}
    
    return block

# generate the proper number of blocks
blocks = []
for b in range(num_blocks):
    blocks.append(make_block())

In [8]:
blocks

[{'study': [{'description': 'avenue',
    'word_no': '646',
    'valence_mean': '5.5',
    'valence_sd': '1.3700000000000001',
    'arousal_mean': '4.1200000000000001',
    'arousal_sd': '2.0099999999999998',
    'dominance_mean': '5.4000000000000004',
    'dominance_sd': '1.53',
    'word_frequency': '46',
    'valence': 'neu',
    'cond': 'once',
    'pres_num': 1,
    'type': 'target'},
   {'description': 'basket',
    'word_no': '547',
    'valence_mean': '5.4500000000000002',
    'valence_sd': '1.1499999999999999',
    'arousal_mean': '3.6299999999999999',
    'arousal_sd': '2.02',
    'dominance_mean': '5.7599999999999998',
    'dominance_sd': '1.45',
    'word_frequency': '17',
    'valence': 'neu',
    'cond': 'refresh',
    'pres_num': 1,
    'type': 'target'},
   {'description': '+',
    'word_no': '547',
    'valence_mean': '5.4500000000000002',
    'valence_sd': '1.1499999999999999',
    'arousal_mean': '3.6299999999999999',
    'arousal_sd': '2.02',
    'dominance_mean': '

In [9]:
# save the blocks out to a pickle file
# (note the 'b' in the 'wb', which means a 
# binary stream instead of a ascii text stream)
pickle.dump(blocks, open('refresh_blocks.pickle', 'wb'))

In [10]:
# show how to read it back in
my_blocks = pickle.load(open('refresh_blocks.pickle','rb'))
my_blocks

[{'study': [{'description': 'avenue',
    'word_no': '646',
    'valence_mean': '5.5',
    'valence_sd': '1.3700000000000001',
    'arousal_mean': '4.1200000000000001',
    'arousal_sd': '2.0099999999999998',
    'dominance_mean': '5.4000000000000004',
    'dominance_sd': '1.53',
    'word_frequency': '46',
    'valence': 'neu',
    'cond': 'once',
    'pres_num': 1,
    'type': 'target'},
   {'description': 'basket',
    'word_no': '547',
    'valence_mean': '5.4500000000000002',
    'valence_sd': '1.1499999999999999',
    'arousal_mean': '3.6299999999999999',
    'arousal_sd': '2.02',
    'dominance_mean': '5.7599999999999998',
    'dominance_sd': '1.45',
    'word_frequency': '17',
    'valence': 'neu',
    'cond': 'refresh',
    'pres_num': 1,
    'type': 'target'},
   {'description': '+',
    'word_no': '547',
    'valence_mean': '5.4500000000000002',
    'valence_sd': '1.1499999999999999',
    'arousal_mean': '3.6299999999999999',
    'arousal_sd': '2.02',
    'dominance_mean': '

In [11]:
import pandas as pd

sl = pd.DataFrame(blocks[0]['study'])
sl

,description,word_no,valence_mean,valence_sd,arousal_mean,arousal_sd,dominance_mean,dominance_sd,word_frequency,valence,cond,pres_num,type
0,avenue,646,5.5,1.3700000000000001,4.1200000000000001,2.0099999999999998,5.4000000000000004,1.53,46,neu,once,1,target
1,basket,547,5.4500000000000002,1.1499999999999999,3.6299999999999999,2.02,5.7599999999999998,1.45,17,neu,refresh,1,target
2,+,547,5.4500000000000002,1.1499999999999999,3.6299999999999999,2.02,5.7599999999999998,1.45,17,neu,refresh,2,target
3,habit,775,4.1100000000000003,1.77,3.9500000000000002,2.1099999999999999,4.2999999999999998,1.79,23,neg,repeat,1,target
4,habit,775,4.1100000000000003,1.77,3.9500000000000002,2.1099999999999999,4.2999999999999998,1.79,23,neg,repeat,2,target
5,dump,733,3.21,1.8700000000000001,4.1200000000000001,2.3599999999999999,3.8300000000000001,1.8700000000000001,4,neg,once,1,target
6,lust,519,7.1200000000000001,1.6200000000000001,6.8799999999999999,1.8500000000000001,5.4900000000000002,2.27,5,pos,repeat,1,target
7,lust,519,7.1200000000000001,1.6200000000000001,6.8799999999999999,1.8500000000000001,5.4900000000000002,2.27,5,pos,repeat,2,target
8,alimony,634,3.9500000000000002,2.0,4.2999999999999998,2.29,4.6299999999999999,2.2999999999999998,2,neg,refresh,1,target
9,+,634,3.9500000000000002,2.0,4.2999999999999998,2.29,4.6299999999999999,2.2999999999999998,2,neg,refresh,2,target


In [12]:
sl.groupby(['cond', 'valence'])['pres_num'].count()

cond     valence
once     neg        1
         neu        1
         pos        1
refresh  neg        2
         neu        2
         pos        2
repeat   neg        2
         neu        2
         pos        2
Name: pres_num, dtype: int64

## SMILE Experiment Details

*Provide enough detail so that someone could implement the experiment presentation and response collection, including all timing information and how the blocks were structured and presented.*

In [13]:
# The condition where there is no wait time between the study and test condition 

from smile.common import *
from smile.scale import scale as s

# config section
font_size = 75
resp_keys = ['O', 'N']
resp_map = {'target': 'O', 'lure': 'N'}
ISI_dur = .5
ISI_jitter = .5
hr_jitter = 3600
study_dur = .5
inst_font_size = 50



inst_text = '[u][size=70]Memory Recognition Test INSTRUCTIONS[/size][/u] \nThis is a recognition memory experiment.  \nThis means you will be presented with a list of words one at a time from a study list. \nAfter a short delay, you will be tested for if you recall seeing that word from the previous words shown in the study list. \nThis means when you are in the test phase of the experiment, you will be given the words from your study list, as well as new words that were not previously seen. \nIn the test phase of each block, participants will see the study items again, along with an equal number of new items,and for each item you must specify whether the item is an old target item (i.e., one that was on the study list) or a new lure item. \n Press ENTER key to continue.'
# create the experiment
exp = Experiment(name='PARTICIPANT', show_splash=False, 
                 fullscreen=False,
                 resolution=(1024, 768), scale_box=(1024, 768))

@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None), 
          center=exp.screen.center, halign='center',
          markup=True, )
    with UntilDone():
        KeyPress(keys=['ENTER'])

    Label(text='In this test you will only need to use two keys: Press the key "O" to indicate the word was on the study list or an "Old" word. \n Press the key "N" to indicate the word was not on the study list or a "New" word. \nPress ENTER key to continue.',
          font_size=font_size,
          text_size=(exp.screen.width*0.75, None), 
          center=exp.screen.center, halign='center',
          markup=True, )
    with UntilDone():
        KeyPress(keys=['ENTER'])

# STUDY TRIAL WORDS APPEAR 
@Subroutine
def StudyList(self, block_num, trial_num, study_trial):
    # present the stimulus
    stim = Label(text=study_trial['description'],  # Use 'description' from study_trial
                 font_size=font_size,
                duration = study_dur)

    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)
    
    #????????????????????????
    # Create a study item log: Tells the researcher when each word appeared on the screen 
       
    # log the result of the trial
    Log(name='participant', 
        log_dict1=study_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=ISI_dur
       )
    
    
# TEST TRIAL WORDS APPEAR 
@Subroutine
def TestList(self, block_num, trial_num, test_trial):
    # present the stimulus
    stim = Label(text=test_trial['description'],  # Use 'description' from test_trial
                 font_size=font_size)
    
    with UntilDone():
        Wait(until=stim.appear_time)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[test_trial['type']])  # Use 'type' from test_trial
    
    # log the result of the trial
    Log(name='participant', 
        log_dict=test_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
       )


# Start of Simulation: 
Label(text='Welcome to the Memory Recognition Test! \nPress ENTER key to Continue',
      font_size=font_size)
with UntilDone():
    KeyPress(keys=['ENTER'])

# Get the subj id information
from smile.startup import InputSubject

InputSubject('Participant')

# show the instructions
Instruct()
Wait(0.5)
    
# loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])

    # add in some delay before the start of the block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # loop over the study trial
    with Loop(block.current['study']) as study_trial:
        StudyList(block.i, study_trial.i, study_trial.current)
    
    # add in some delay before the start of the test block (two conditions)
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # OR 
    #Wait(ISI_dur, jitter=hr_jitter)

    # loop over the test trial
    with Loop(block.current['test']) as test_trial:
        TestList(block.i, test_trial.i, test_trial.current)
    

# make sure they are ready to continue
Label(text='You are all done!!!\nPress the ENTER key to go celebrate.', 
      font_size=font_size, halign='center')
with UntilDone():
    KeyPress(keys=['ENTER'])

# run the experiment
exp.run()

In [14]:
# The condition where there is an hour wait time between the study list and the test list  

from smile.common import *
from smile.scale import scale as s

# config section
font_size = 75
resp_keys = ['O', 'N']
resp_map = {'target': 'O', 'lure': 'N'}
ISI_dur = .5
ISI_jitter = .5
hr_jitter = 3600
study_dur = .5
inst_font_size = 50



inst_text = '[u][size=70]Memory Recognition Test INSTRUCTIONS[/size][/u] \nThis is a recognition memory experiment.  \nThis means you will be presented with a list of words one at a time from a study list. \nAfter a short delay, you will be tested for if you recall seeing that word from the previous words shown in the study list. \nThis means when you are in the test phase of the experiment, you will be given the words from your study list, as well as new words that were not previously seen. \nIn the test phase of each block, participants will see the study items again, along with an equal number of new items,and for each item you must specify whether the item is an old target item (i.e., one that was on the study list) or a new lure item. \n Press ENTER key to continue.'
# create the experiment
exp = Experiment(name='PARTICIPANT', show_splash=False, 
                 fullscreen=False,
                 resolution=(1024, 768), scale_box=(1024, 768))

@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None), 
          center=exp.screen.center, halign='center',
          markup=True, )
    with UntilDone():
        KeyPress(keys=['ENTER'])

    Label(text='In this test you will only need to use two keys: Press the key "O" to indicate the word was on the study list or an "Old" word. \n Press the key "N" to indicate the word was not on the study list or a "New" word. \nPress ENTER key to continue.',
          font_size=font_size,
          text_size=(exp.screen.width*0.75, None), 
          center=exp.screen.center, halign='center',
          markup=True, )
    with UntilDone():
        KeyPress(keys=['ENTER'])

# STUDY TRIAL WORDS APPEAR 
@Subroutine
def StudyList(self, block_num, trial_num, study_trial):
    # present the stimulus
    stim = Label(text=study_trial['description'],  # Use 'description' from study_trial
                 font_size=font_size,
                duration = study_dur)

    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)
    
    #????????????????????????
    # Create a study item log: Tells the researcher when each word appeared on the screen 
       
    # log the result of the trial
    Log(name='participant', 
        log_dict1=study_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=ISI_dur
       )
    
    
# TEST TRIAL WORDS APPEAR 
@Subroutine
def TestList(self, block_num, trial_num, test_trial):
    # present the stimulus
    stim = Label(text=test_trial['description'],  # Use 'description' from test_trial
                 font_size=font_size)
    
    with UntilDone():
        Wait(until=stim.appear_time)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[test_trial['type']])  # Use 'type' from test_trial
    
    # log the result of the trial
    Log(name='participant', 
        log_dict=test_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
       )


# Start of Simulation: 
Label(text='Welcome to the Memory Recognition Test! \nPress ENTER key to Continue',
      font_size=font_size)
with UntilDone():
    KeyPress(keys=['ENTER'])

# Get the subj id information
from smile.startup import InputSubject

InputSubject('Participant')

# show the instructions
Instruct()
Wait(0.5)
    
# loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])

    # add in some delay before the start of the block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # loop over the study trial
    with Loop(block.current['study']) as study_trial:
        StudyList(block.i, study_trial.i, study_trial.current)
    
    # add in some delay before the start of the test block (two conditions)
    # Wait(ISI_dur, jitter=ISI_jitter)
    
    # OR 
    Wait(ISI_dur, jitter=hr_jitter)

    # loop over the test trial
    with Loop(block.current['test']) as test_trial:
        TestList(block.i, test_trial.i, test_trial.current)
    

# make sure they are ready to continue
Label(text='You are all done!!!\nPress the ENTER key to go celebrate.', 
      font_size=font_size, halign='center')
with UntilDone():
    KeyPress(keys=['ENTER'])

# run the experiment
exp.run()

# Results

*In this section, state a specific question, then define your dependent and independent variables that will help you answer that question. As stated above, your question must give rise to an analysis that is not identical to one we performed in class (i.e., you must do more than copy and paste code with zero changes. That said, the analysis can match those from the class quite closely.*

*Note: You need to repeat this process for a total of two hypotheses/questions.*

## Data processing and visualization

*With the lessons as a guide, process your data to create the necessary data frame to plot the visualization associated with the question stated above. Then plot those data.*

In [1]:
# Code for data processing and visualization

In [1]:
# custom function to load slogs
def load_all_subj_logs(data_dir, log_file):
    # load in a list of all the subj
    subjs = [os.path.splitext(os.path.split(filepath)[1])[0].split('_')[-1]
             for filepath in glob(os.path.join(data_dir, log_file + '*.slog'))]
    subjs.sort()

    # loop over subj and their data
    all_dat = []
    for subj in subjs:
        # set the file
        log_path = os.path.join(data_dir, log_file+'_'+subj+'.slog')
        #print(log_path)

        # load the data
        try:
            all_dat.extend(log2dl(log_path, subj=subj))
        except:
            pass

    df = pd.DataFrame(all_dat)
    
    return df

*Put text here describing what is in your plot like a detailed figure caption.*

## Statistical test and interpretation

*Perform a statistical test to support your conclusions with regard to your question outlined above. This can be with either statsmodels or with bambi.*

In [3]:
# Code for statistical test (can be either with statsmodels or bambi)

*Put text here describing the results of your statistical test*